In [2]:
import astropy
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import subprocess
import os

In [95]:
def pop_list(ls,indices):
    for n,ind in enumerate(indices):
        ls.pop(ind)
    return(ls)

In [3]:
photz = Table.read('data/photz/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_hpx_public.fits')

In [23]:
photz[0]

['ID',
 'ID_OPTICAL',
 'ID_SPITZER',
 'RA',
 'DEC',
 'help_id',
 'RA_HELP',
 'DEC_HELP',
 'EBV',
 'Z_SPEC',
 'Z_SOURCE',
 'Z_QUAL',
 'RA_ZSPEC',
 'DEC_ZSPEC',
 'REL',
 'OBJID',
 'CLASS_STAR',
 'FLAG_DEEP',
 'FLAG_OVERLAP',
 'FLAG_CLEAN',
 'FUV_flux',
 'FUV_fluxerr',
 'NUV_flux',
 'NUV_fluxerr',
 'u_flux',
 'u_fluxerr',
 'Bw_flux',
 'Bw_fluxerr',
 'R_flux',
 'R_fluxerr',
 'I_flux',
 'I_fluxerr',
 'z_flux',
 'z_fluxerr',
 'z_Subaru_flux',
 'z_Subaru_fluxerr',
 'y_flux',
 'y_fluxerr',
 'J_flux',
 'J_fluxerr',
 'H_flux',
 'H_fluxerr',
 'K_flux',
 'K_fluxerr',
 'Ks_flux',
 'Ks_fluxerr',
 'ch1_flux',
 'ch1_fluxerr',
 'ch2_flux',
 'ch2_fluxerr',
 'ch3_flux',
 'ch3_fluxerr',
 'ch4_flux',
 'ch4_fluxerr',
 'F_MIPS_24',
 'FErr_MIPS_24_u',
 'FErr_MIPS_24_l',
 'Bkg_MIPS_24',
 'Sig_conf_MIPS_24',
 'Rhat_MIPS_24',
 'n_eff_MIPS_24',
 'Pval_res_24',
 'flag_mips_24',
 'F_PACS_100',
 'FErr_PACS_100_u',
 'FErr_PACS_100_l',
 'F_PACS_160',
 'FErr_PACS_160_u',
 'FErr_PACS_160_l',
 'Bkg_PACS_100',
 'Bkg_PACS_

In [138]:
cigale_input = photz.copy()

In [135]:
filters_file = open('data/photz/filters/filter.bootes_mbrown_2014a.res.info','r')
filters = []
for line in filters_file:
    temp = line.split(' ')
    filters.append(temp)
filters_file.close()

for n,line in enumerate(filters):
    if n!=32:
        filters[n] = line[0:-1]
    else:
        filters[n][-1] = filters[n][-1].replace('\n','')
    filters[n] = pop_list(filters[n],[1 for m in range(len(filters[n])-2)])
    filters[n][-1] = filters[n][-1].replace('.filter','')

In [129]:
translate_file = open('data/photz/filters/brown.zphot.2014.translate','r')
translate = []
for line in translate_file:
    if '#' in line:
        continue
    temp = line.split(' ')
    translate.append(temp)
translate_file.close()

translate = translate[:len(translate)-1]

for n,line in enumerate(translate):
    translate[n][1] = line[1][1:-1]

In [139]:
for n,line in enumerate(translate):
    colname = line[0]
    filt_num = int(line[1])
    filt_name = ''
    for m,filt in enumerate(filters):
        if m+1==filt_num:
            print(filt)
            filt_name = filt[1]
    
    if 'err' in colname:
        filt_name = filt_name+'_err'
    #print(colname)
    #print(filt_name)
    cigale_input.rename_column(colname,filt_name)

['9', 'LBCBlue_Uspec']
u_flux
LBCBlue_Uspec
['9', 'LBCBlue_Uspec']
u_fluxerr
LBCBlue_Uspec_err
['2', 'Bw_atmccd']
Bw_flux
Bw_atmccd
['2', 'Bw_atmccd']
Bw_fluxerr
Bw_atmccd_err
['11', 'R_atmccd']
R_flux
R_atmccd
['11', 'R_atmccd']
R_fluxerr
R_atmccd_err
['7', 'I_atmccd']
I_flux
I_atmccd
['7', 'I_atmccd']
I_fluxerr
I_atmccd_err
['34', 'sdssz2']
z_flux
sdssz2
['34', 'sdssz2']
z_fluxerr
sdssz2_err
['33', 'suprime_FDCCD_z']
z_Subaru_flux
suprime_FDCCD_z
['33', 'suprime_FDCCD_z']
z_Subaru_fluxerr
suprime_FDCCD_z_err
['10', 'LBCR_Y']
y_flux
LBCR_Y
['10', 'LBCR_Y']
y_fluxerr
LBCR_Y_err
['19', 'jband']
J_flux
jband
['19', 'jband']
J_fluxerr
jband_err
['18', 'hband']
H_flux
hband
['18', 'hband']
H_fluxerr
hband_err
['8', 'K_atmccd']
K_flux
K_atmccd
['8', 'K_atmccd']
K_fluxerr
K_atmccd_err
['20', 'kband']
Ks_flux
kband
['20', 'kband']
Ks_fluxerr
kband_err
['3', 'IRAC_ch1_total_response']
ch1_flux
IRAC_ch1_total_response
['3', 'IRAC_ch1_total_response']
ch1_fluxerr
IRAC_ch1_total_response_err
['4'

In [140]:
cigale_input.rename_column('z1_median','redshift')

ID,ID_OPTICAL,ID_SPITZER,RA,DEC,help_id,RA_HELP,DEC_HELP,EBV,Z_SPEC,Z_SOURCE,Z_QUAL,RA_ZSPEC,DEC_ZSPEC,REL,OBJID,CLASS_STAR,FLAG_DEEP,FLAG_OVERLAP,FLAG_CLEAN,FUV_flux,FUV_fluxerr,NUV_flux,NUV_fluxerr,LBCBlue_Uspec,LBCBlue_Uspec_err,Bw_atmccd,Bw_atmccd_err,R_atmccd,R_atmccd_err,I_atmccd,I_atmccd_err,sdssz2,sdssz2_err,suprime_FDCCD_z,suprime_FDCCD_z_err,LBCR_Y,LBCR_Y_err,jband,jband_err,hband,hband_err,K_atmccd,K_atmccd_err,kband,kband_err,IRAC_ch1_total_response,IRAC_ch1_total_response_err,IRAC_ch2_total_response,IRAC_ch2_total_response_err,IRAC_ch3_total_response,IRAC_ch3_total_response_err,IRAC_ch4_total_response,IRAC_ch4_total_response_err,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24,F_PACS_100,FErr_PACS_100_u,FErr_PACS_100_l,F_PACS_160,FErr_PACS_160_u,FErr_PACS_160_l,Bkg_PACS_100,Bkg_PACS_160,Sig_conf_PACS_100,Sig_conf_PACS_160,Rhat_PACS_100,Rhat_PACS_160,n_eff_PACS_100,n_eff_PACS_160,Pval_res_100,Pval_res_160,flag_PACS_100,flag_PACS_160,F_SPIRE_250,FErr_SPIRE_250_u,FErr_SPIRE_250_l,F_SPIRE_350,FErr_SPIRE_350_u,FErr_SPIRE_350_l,F_SPIRE_500,FErr_SPIRE_500_u,FErr_SPIRE_500_l,Bkg_SPIRE_250,Bkg_SPIRE_350,Bkg_SPIRE_500,Sig_conf_SPIRE_250,Sig_conf_SPIRE_350,Sig_conf_SPIRE_500,Rhat_SPIRE_250,Rhat_SPIRE_350,Rhat_SPIRE_500,n_eff_SPIRE_250,n_eff_SPIRE_500,n_eff_SPIRE_350,Pval_res_250,Pval_res_350,Pval_res_500,flag_spire_250,flag_spire_350,flag_spire_500,AGN,optAGN,IRAGN,AGN_ZSPEC,XrayAGN,XrayFlux_0.5-2,XrayHardness,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,nfilt_eazy,nfilt_atlas,nfilt_ananna,chi_r_best,chi_r_stellar,stellar_type,hp_idx_11,hp_depth_cluster
,,,deg,deg,,deg,deg,,,,,,,,,,,,,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,muJy,muJy,muJy,MJy / sr,MJy / sr,,,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,,,,,,,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,
int64,int64,int64,float64,float64,bytes27,float64,float64,float32,float64,int64,int64,float64,float64,float64,bytes25,float32,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,bool,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,bytes6,int64,int32
0,1,--,216.473863,32.2825963,N/A,nan,nan,0.01093379,nan,--,--,nan,nan,nan,N/A,0.64,0,0,1,-99.0,-99.0,-99.0,-99.0,0.10352171485738193,0.09050546420954841,-99.0,-99.0,-99.0,-99.0,1.2489124884509977,0.17990861652974743,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-0.49297147310738154,0.9599805695282839,0.20943547943156204,1.134281811727078,7.514107605812171,4.5540334671514415,7.615719616676927,4.989334194948697,nan,nan,nan,nan,nan,nan,nan,nan,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,True,0,0.0,0.0,--,0.0,-99.0,-99.0,3.6887,2.88,4.63,0.379,0.3205,0.0,0.81,0.3,6,6,-99,0.9076026,-99.0,,9355052,38
1,2,--,216.6176098,32.2827433,N/A,nan,nan,0.011378967,nan,--,--,nan,nan,nan,N/A,0.66,1,0,1,-99.0,-99.0,-99.0,-99.0,0.14205340354133433,0.102083831

In [ ]:
Table.write(cigale_input,'cigale_input_Lockman.fits',format='fits',overwrite=True)